# **Análise de Atrasos em Voos nos EUA**

Este projeto tem como objetivo construir um pipeline de dados utilizando a plataforma Databricks e ferramentas locais como Python, Jupyter e GitHub. A proposta é investigar padrões de atrasos aéreos nos Estados Unidos a partir de um conjunto de dados público disponibilizado pelo Kaggle. 

A análise será orientada pelas perguntas de negócio previamente definidas no README e executada de forma documentada, com foco em atender todos os critérios de avaliação propostos no desenvolvimento do MVP.

Todas as etapas do pipeline — da coleta até a análise — serão comentadas em primeira pessoa para guiar a banca avaliadora no raciocínio por trás de cada decisão.


In [7]:
# Importação inicial das bibliotecas essenciais para análise
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações de exibição
pd.set_option('display.max_columns', 100)
sns.set(style="whitegrid")


## **Carregamento dos Dados**

Aqui estou realizando o carregamento inicial do arquivo `flights.csv`, extraído do conjunto de dados público do Kaggle e armazenado localmente na pasta `data/`. 

Caso o arquivo esteja muito grande, futuramente farei uma amostragem para facilitar o desenvolvimento e execução.

## **Redução do Dataset Original**

O arquivo original `flights.csv`, com aproximadamente 580 MB, foi utilizado apenas localmente para gerar uma amostra representativa do conjunto de dados. Para garantir que o projeto possa ser executado diretamente a partir do GitHub, incluí a versão reduzida `flights_amostra.csv`, que contém 20.000 registros aleatórios do dataset original.

Esta amostragem permite realizar análises sem comprometer a performance, mantendo a representatividade dos dados.

Abaixo está o código utilizado para gerar a amostra:



In [10]:
# Carregando o dataset principal
df = pd.read_csv('data/flights.csv', low_memory=False)

# Gerar uma amostra de 20.000 linhas aleatórias (reprodutível com random_state)
amostra = df.sample(n=20000, random_state=42)
amostra.to_csv("data/flights_amostra.csv", index=False)

# Visualizando as primeiras linhas
amostra.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
1508570,2015,4,7,2,EV,4900,N759EV,FWA,DTW,1340,1335.0,-5.0,13.0,1348.0,56.0,48.0,30.0,128,1418.0,5.0,1436,1423.0,-13.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
363270,2015,1,24,6,AS,611,N413AS,LAS,SEA,1910,1858.0,-12.0,14.0,1912.0,155.0,155.0,131.0,867,2123.0,10.0,2145,2133.0,-12.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3003945,2015,7,8,3,WN,1483,N463WN,OAK,SEA,630,626.0,-4.0,8.0,634.0,110.0,106.0,92.0,672,806.0,6.0,820,812.0,-8.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2291425,2015,5,26,2,WN,193,N7745A,STL,DAL,810,1043.0,153.0,6.0,1049.0,100.0,99.0,85.0,546,1214.0,8.0,950,1222.0,152.0,0,0,NaN,0.0,0.0,0.0,152.0,0.0
2973084,2015,7,6,1,UA,253,N213UA,IAH,HNL,1000,1008.0,8.0,16.0,1024.0,498.0,488.0,461.0,3904,1305.0,11.0,1318,1316.0,-2.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


## **Dimensões do Conjunto de Dados e Tipos de Variáveis**

Abaixo verifico a quantidade de linhas e colunas, além dos tipos de dados presentes. Isso me ajuda a entender a complexidade e possíveis etapas de tratamento necessárias.


In [13]:
print(f"Número de linhas: {amostra.shape[0]}")
print(f"Número de colunas: {amostra.shape[1]}")

print("\nInfo Amostra:")
amostra.info()

Número de linhas: 20000
Número de colunas: 31

Info Amostra:
<class 'pandas.core.frame.DataFrame'>
Index: 20000 entries, 1508570 to 2046378
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   YEAR                 20000 non-null  int64  
 1   MONTH                20000 non-null  int64  
 2   DAY                  20000 non-null  int64  
 3   DAY_OF_WEEK          20000 non-null  int64  
 4   AIRLINE              20000 non-null  object 
 5   FLIGHT_NUMBER        20000 non-null  int64  
 6   TAIL_NUMBER          19948 non-null  object 
 7   ORIGIN_AIRPORT       20000 non-null  object 
 8   DESTINATION_AIRPORT  20000 non-null  object 
 9   SCHEDULED_DEPARTURE  20000 non-null  int64  
 10  DEPARTURE_TIME       19722 non-null  float64
 11  DEPARTURE_DELAY      19722 non-null  float64
 12  TAXI_OUT             19708 non-null  float64
 13  WHEELS_OFF           19708 non-null  float64
 14  SCHEDULED_TIME       2